In [1]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *
from process import *

In [2]:
# データ保存先
data_folda = '../data/'

In [3]:
# make_timeseries_data.ipynb で作成したデータを読み込む
pseudo_time_series=pd.read_pickle(data_folda+'pseudo_time_series.pkl')

In [4]:
pseudo_time_series.columns

Index(['restaurant_id', 'original_id', 'available', 'closed_date',
       'business_status', 'prefecture', 'base_date', 'closed_flg', 'target3',
       'target6', 'eval_period'],
      dtype='object')

In [5]:
tabelog = pd.read_csv(data_folda+'20231121163030_DG-Lab_tabelog_restaurant_data.csv')

/tmp/ipykernel_12193/243680395.py:1: DtypeWarning: Columns (28,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  tabelog = pd.read_csv(data_folda+'20231121163030_DG-Lab_tabelog_restaurant_data.csv')


### master data から加えるデータを定義

In [6]:
add_list=['name','branch_name','opened_date', 'created_at', 'genres',"latitude","longitude",
          "net_reservation_flg",
          'price_range_lunch_by_owner','price_range_dinner_by_owner', 'price_range_lunch_by_review','price_range_dinner_by_review']

# カウント系のカラムを追加
count_columns=tabelog.columns[tabelog.columns.str.contains('count')].tolist()

add_list+=count_columns

* count 系のカラムも追加

count_columns=tabelog.columns[tabelog.columns.str.contains('count')].tolist()


In [7]:
# augumented_data から不要なカラムを削除
# 不要なカラム
# 'list_type_id','telephone', 'reservation_telephone', 'url', 'original_id', 'available', 'sort_no', 'created', 'modified'
pseudo_time_series.drop(['available'], axis=1, inplace=True)

In [8]:
pseudo_time_series=pd.merge(pseudo_time_series, tabelog[["id"]+add_list], left_on="original_id",right_on='id', how='left')

### 営業日の計算。* 本来は opened_date と created at の古い方を採用したいが、opened_date が古すぎて datetime型に変換出来ないので、created at のみで対応。 

In [9]:
pseudo_time_series["created_at"]=pd.to_datetime(pseudo_time_series["created_at"])


In [10]:
"""
#pseudo_time_series["opened_date"]=pd.to_datetime(pseudo_time_series["opened_date"])

pseudo_time_series["opened_date_temp"]=pseudo_time_series["opened_date"]
# Define minimum and maximum Timestamp values
min_date = pd.Timestamp.min
max_date = pd.Timestamp.max

# if opemed_date is less than min_date, replace it with min_date
pseudo_time_series['opened_date_temp'] = pseudo_time_series['opened_date_temp'].apply(lambda x: min_date if pd.isnull(x) and pd.to_datetime(x, errors='ignore') < min_date else x)
# if opemed_date is greater than max_date, replace it with max_date
pseudo_time_series['opened_date_temp'] = pseudo_time_series['opened_date_temp'].apply(lambda x: max_date if pd.isnull(x) and pd.to_datetime(x, errors='ignore') > max_date else x)

# Convert 'date' column to datetime, setting out-of-bounds dates to NaT
pseudo_time_series['opened_date_temp'] = pd.to_datetime(pseudo_time_series['opened_date_temp'], errors='coerce')

print(pseudo_time_series['opened_date_temp'])
"""


'\n#pseudo_time_series["opened_date"]=pd.to_datetime(pseudo_time_series["opened_date"])\n\npseudo_time_series["opened_date_temp"]=pseudo_time_series["opened_date"]\n# Define minimum and maximum Timestamp values\nmin_date = pd.Timestamp.min\nmax_date = pd.Timestamp.max\n\n# if opemed_date is less than min_date, replace it with min_date\npseudo_time_series[\'opened_date_temp\'] = pseudo_time_series[\'opened_date_temp\'].apply(lambda x: min_date if pd.isnull(x) and pd.to_datetime(x, errors=\'ignore\') < min_date else x)\n# if opemed_date is greater than max_date, replace it with max_date\npseudo_time_series[\'opened_date_temp\'] = pseudo_time_series[\'opened_date_temp\'].apply(lambda x: max_date if pd.isnull(x) and pd.to_datetime(x, errors=\'ignore\') > max_date else x)\n\n# Convert \'date\' column to datetime, setting out-of-bounds dates to NaT\npseudo_time_series[\'opened_date_temp\'] = pd.to_datetime(pseudo_time_series[\'opened_date_temp\'], errors=\'coerce\')\n\nprint(pseudo_time_seri

In [11]:
pseudo_time_series["real_opened_date"]=pd.to_datetime(pseudo_time_series["created_at"])
# opoen_date がある場合は、それを使う
# pseudo_time_series.loc[pseudo_time_series["opened_date"].notnull(),"real_opened_date"]=pd.to_datetime(pseudo_time_series["opened_date"], format='%Y-%m-%d', errors='coerce')

# "created_at" と yyyymmdd の差分を計算
pseudo_time_series["openning_days"] = pd.to_datetime(pseudo_time_series["base_date"])-pd.to_datetime(pseudo_time_series["real_opened_date"], errors='coerce')

* price range をダミー化

In [12]:
p_list=['price_range_lunch_by_owner','price_range_dinner_by_owner', 'price_range_lunch_by_review','price_range_dinner_by_review']
for p in p_list:
    pseudo_time_series=get_price_range_dummy(pseudo_time_series,p)

～￥999
￥1,000～￥1,999
￥2,000～￥2,999
￥3,000～￥3,999
￥4,000～￥4,999
￥5,000～￥5,999
￥6,000～￥7,999
￥8,000～￥9,999
￥10,000～￥14,999
￥15,000～￥19,999
￥20,000～￥29,999
￥30,000～￥39,999
￥40,000～￥49,999
￥50,000～￥59,999
￥60,000～￥79,999
￥80,000～￥99,999
￥100,000～
～￥999
￥1,000～￥1,999
￥2,000～￥2,999
￥3,000～￥3,999
￥4,000～￥4,999
￥5,000～￥5,999
￥6,000～￥7,999
￥8,000～￥9,999
￥10,000～￥14,999
￥15,000～￥19,999
￥20,000～￥29,999
￥30,000～￥39,999
￥40,000～￥49,999
￥50,000～￥59,999
￥60,000～￥79,999
￥80,000～￥99,999
￥100,000～
～￥999
￥1,000～￥1,999
￥2,000～￥2,999
￥3,000～￥3,999
￥4,000～￥4,999
￥5,000～￥5,999
￥6,000～￥7,999
￥8,000～￥9,999
￥10,000～￥14,999
￥15,000～￥19,999
￥20,000～￥29,999
￥30,000～￥39,999
￥40,000～￥49,999
￥50,000～￥59,999
￥60,000～￥79,999
￥80,000～￥99,999
￥100,000～
～￥999
￥1,000～￥1,999
￥2,000～￥2,999
￥3,000～￥3,999
￥4,000～￥4,999
￥5,000～￥5,999
￥6,000～￥7,999
￥8,000～￥9,999
￥10,000～￥14,999
￥15,000～￥19,999
￥20,000～￥29,999
￥30,000～￥39,999
￥40,000～￥49,999
￥50,000～￥59,999
￥60,000～￥79,999
￥80,000～￥99,999
￥100,000～


In [ ]:
# base_dateがcreated_atよりも前の場合は eval_period を False にする
pseudo_time_series.loc[pseudo_time_series["base_date"]<pseudo_time_series["created_at"],"eval_period"]=False

In [ ]:
pseudo_time_series.to_pickle(data_folda+'pseudo_time_series_with_master.pkl')